<a href="https://colab.research.google.com/github/nijatmaharramov/Computer-Vision-Projects/blob/main/Fruit_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fruits and Vegetables Image Recognition Dataset

# Importing libraries and creating paths to datasets#

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition

In [ ]:
!unzip fruit-and-vegetable-image-recognition.zip

In [ ]:
for dirpath, dirnames, filenames, in os.walk('/content/train'):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
import tensorflow as tf

train_data_path = '/content/train'


train_data = tf.keras.utils.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    interpolation='nearest',
    batch_size=32)

class_names = train_data.class_names
print(class_names)

In [ ]:
test_data_path = '/content/test'

test_data = tf.keras.utils.image_dataset_from_directory(
    test_data_path,
    labels = 'inferred',
    label_mode = 'categorical',
    image_size=(224,224),
    interpolation='nearest',
    batch_size=32
)

In [ ]:
validation_data_path = '/content/validation'

validation_data = tf.keras.utils.image_dataset_from_directory(
    validation_data_path,
    labels = 'inferred',
    label_mode = 'categorical',
    image_size=(224,224),
    interpolation='nearest',
    batch_size=32
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 15))
for images, labels in train_data.take(1):
  for i in range(12):
    ax = plt.subplot(6, 6, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i].numpy().argmax()])
    plt.axis("off")

# Our first model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (224,224,3)),
    tf.keras.layers.Rescaling(1 / 255.),
    tf.keras.layers.Conv2D(64, kernel_size = 7, padding='same', activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, kernel_size = 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(128, kernel_size = 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, kernel_size = 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Conv2D(256, kernel_size = 3, padding='same', activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax', kernel_initializer='glorot_normal')
    ])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy', 'f1_score'])

model.fit(train_data, validation_data=validation_data, epochs=5 )

# Using Pre-trained Model

In [ ]:
inputs = tf.keras.layers.Input(shape = (224, 224, 3))

x = tf.keras.applications.resnet50.preprocess_input(inputs)



In [ ]:
pre_trained_model = tf.keras.applications.ResNet50(
    input_shape=(224,224,3),
    input_tensor=x,
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

In [ ]:
for layer in pre_trained_model.layers:
    print(layer.name)

In [ ]:
pre_trained_model.trainable = False

In [ ]:
model2 = tf.keras.Sequential([
    pre_trained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

In [ ]:
tf.keras.utils.plot_model(model2, show_shapes=True, show_layer_names=False, dpi=100)


In [ ]:
model2.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               metrics=['accuracy', 'f1_score'])

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("fruit_detector.keras", save_best_only = True)

early_stp = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# ReduceLROnPlateau - it reduces lr 0.5 times if there is no positive progress on val_loss for 3 epochs
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [ ]:
history = model2.fit(train_data, validation_data=validation_data, epochs=10, callbacks=[checkpoint_cb, early_stp])

# Predictive system

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def plot_and_pred(model, image_path):
    plt.figure(figsize = (12,12))
    image = mpimg.imread(image_path)
    image = tf.image.resize(image, (224,224))
    pred_probs = model.predict(np.expand_dims(image, axis=0))
    pred = class_names[np.argmax(pred_probs)]
    plt.subplot(1, 2, 1)
    plt.imshow(image / 255.)
    plt.title(pred, color = 'green')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.bar(class_names, pred_probs[0])
    plt.xlabel('Class')
    plt.ylabel('Probability')
    plt.xticks(rotation=90)


In [ ]:
plot_and_pred(model2, '/content/red-apple-freshleaf-dubai-uae-img01.jpg')

In [ ]:
plot_and_pred(model2, '/content/Watermelon-cuts-24-1.jpg')